In [1]:
import streamlit as st
import time
import re
import random
from pymongo.mongo_client import MongoClient
import pandas
import json
import random
import tqdm as tqdm

In [2]:
RANDOMIZE_CONDITIONS = True
SHUFFLE_SCENARIOS_ORDER = False

nparticipants = 300

scenarios = ['BA', 'IP', 'CJ']
conditions = ['NEUT', 'SQB1', 'SQB2', 'SQB3', 'SQB4']

In [3]:
#update the server collection with default data
uri = st.secrets["mongo"]['uri']
client = MongoClient(uri)
db = client["Replication"]
collexp_code = db.exp_code

In [6]:
#each participant will get one random condition for all three scenarios 
nconditions = len(conditions)
nscenarios = len(scenarios)
nbatches = nparticipants/nconditions

batches = []
codelist = []

for i in range(int(nbatches)):
    bls = []
    for scene in scenarios:
        lbl = [scene+'$'+condition for condition in conditions]
        if RANDOMIZE_CONDITIONS:
            random.shuffle(lbl)
        bls.append(lbl)

    patA = [bls[0][0], bls[1][0], bls[2][0]]
    patB = [bls[0][1], bls[1][1], bls[2][1]]
    patC = [bls[0][2], bls[1][2], bls[2][2]]
    patD = [bls[0][3], bls[1][3], bls[2][3]]
    patE = [bls[0][4], bls[1][4], bls[2][4]]


    if SHUFFLE_SCENARIOS_ORDER:
        random.shuffle(patA)
        random.shuffle(patB)
        random.shuffle(patD)
        random.shuffle(patC)
        random.shuffle(patE)

    patA = '&'.join(patA)
    patB = '&'.join(patB)
    patD = '&'.join(patD)
    patC = '&'.join(patC)
    patE = '&'.join(patE)



    batches.append([patA, patB, patC, patD, patE])
    codelist += [patA, patB, patC, patD, patE]

#index batchlist
idcodelist = [ (str(i+1)+'_'+code, 'EMPTY') for i, code in enumerate(codelist)]

#to json
# Convert nested list to JSON string
json_string = json.dumps(dict(idcodelist))  
records = json.loads(json_string)

In [7]:
records

{'1_BA$SQB2&IP$SQB4&CJ$NEUT': 'EMPTY',
 '2_BA$SQB3&IP$SQB2&CJ$SQB3': 'EMPTY',
 '3_BA$NEUT&IP$SQB3&CJ$SQB4': 'EMPTY',
 '4_BA$SQB1&IP$NEUT&CJ$SQB1': 'EMPTY',
 '5_BA$SQB4&IP$SQB1&CJ$SQB2': 'EMPTY',
 '6_BA$SQB1&IP$SQB3&CJ$SQB1': 'EMPTY',
 '7_BA$SQB4&IP$SQB1&CJ$SQB4': 'EMPTY',
 '8_BA$NEUT&IP$SQB4&CJ$NEUT': 'EMPTY',
 '9_BA$SQB3&IP$SQB2&CJ$SQB3': 'EMPTY',
 '10_BA$SQB2&IP$NEUT&CJ$SQB2': 'EMPTY',
 '11_BA$SQB1&IP$SQB4&CJ$SQB3': 'EMPTY',
 '12_BA$SQB4&IP$NEUT&CJ$SQB4': 'EMPTY',
 '13_BA$SQB2&IP$SQB1&CJ$SQB1': 'EMPTY',
 '14_BA$NEUT&IP$SQB3&CJ$SQB2': 'EMPTY',
 '15_BA$SQB3&IP$SQB2&CJ$NEUT': 'EMPTY',
 '16_BA$SQB3&IP$SQB3&CJ$SQB2': 'EMPTY',
 '17_BA$NEUT&IP$SQB4&CJ$SQB3': 'EMPTY',
 '18_BA$SQB4&IP$NEUT&CJ$SQB1': 'EMPTY',
 '19_BA$SQB2&IP$SQB1&CJ$SQB4': 'EMPTY',
 '20_BA$SQB1&IP$SQB2&CJ$NEUT': 'EMPTY',
 '21_BA$SQB1&IP$SQB3&CJ$SQB3': 'EMPTY',
 '22_BA$SQB4&IP$SQB2&CJ$SQB2': 'EMPTY',
 '23_BA$NEUT&IP$SQB1&CJ$NEUT': 'EMPTY',
 '24_BA$SQB2&IP$NEUT&CJ$SQB4': 'EMPTY',
 '25_BA$SQB3&IP$SQB4&CJ$SQB1': 'EMPTY',
 '26_BA$S

In [11]:
# drop all records
collexp_code.drop()

In [12]:

for record in tqdm.tqdm(records):
    collexp_code.insert_one(dict([[str(record),records[record]]]))

100%|██████████| 300/300 [00:14<00:00, 20.38it/s]
